In [1]:
from sklearn import datasets


iris = datasets.load_iris()
X = iris.data
y = iris.target

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [5]:
import sys
sys.path.append("../") 

# No Uncertainty

In [63]:
import mxnet as mx
import numpy as np
from mxnet import ndarray as nd
from hamiltonian.inference.sgd import sgd
from hamiltonian.models.softmax import softmax

model_ctx=mx.cpu()
hyper={'alpha':.5}
in_units=X.shape[1]
out_units=len(np.unique(y))
model=softmax(hyper,in_units,out_units,ctx=model_ctx)
inference=sgd(model,model.par,step_size=0.01,ctx=model_ctx)

In [64]:
par,loss=inference.fit(epochs=1000,batch_size=60,gamma=0.9,
                       X_train=X_train,y_train=y_train,verbose=True)

  3%|▎         | 31/1000 [00:00<00:06, 153.86it/s]

loss: 0.0092


 13%|█▎        | 128/1000 [00:00<00:05, 150.53it/s]

loss: 0.0030


 21%|██▏       | 214/1000 [00:01<00:06, 121.06it/s]

...

 82%|████████▏ | 820/1000 [00:05<00:01, 136.57it/s]

loss: 0.0019


 91%|█████████▏| 914/1000 [00:06<00:00, 109.27it/s]

loss: 0.0019


100%|██████████| 1000/1000 [00:07<00:00, 135.03it/s]


In [59]:
import hamiltonian
import importlib

try:
    importlib.reload(hamiltonian.models.softmax)
    importlib.reload(hamiltonian.inference.sgd)
    print('modules re-loaded')
except:
    print('no modules loaded yet')

modules re-loaded


In [60]:
model.net.collect_params()

{'0.weight': Parameter (shape=(3, 4), dtype=float32),
 '0.bias': Parameter (shape=(3,), dtype=float32)}

In [61]:
y_pred=model.predict(model.par,nd.array(X_test,ctx=model_ctx))

In [62]:
from sklearn.metrics import classification_report

print(classification_report(np.int32(y_test),np.int32(y_pred.sample().asnumpy())))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       0.78      0.88      0.82         8
           2       0.92      0.86      0.89        14

    accuracy                           0.90        30
   macro avg       0.90      0.91      0.90        30
weighted avg       0.90      0.90      0.90        30



# Aleatoric Uncertainty

In [67]:
X.std(axis=0)

array([5.84333333, 3.05733333, 3.758     , 1.19933333])